In [ ]:
from src.utils import get_vocabulary
from dataclasses import dataclass, field
from typing import Iterator


In [185]:
vocabulary = get_vocabulary(top_n=50000)
print(vocabulary)

{'alarms', 'formally', 'tomkins', 'planks', 'rio', 'burmese', 'hopelessly', 'saffron', 'harriet', 'simile', 'monilia', 'skyjacked', 'exporters', 'huts', 'oriented', 'monopolies', 'pleads', 'welding', 'simon', 'credible', 'remarks', 'homing', 'budzyn', 'restoring', 'dressings', 'parquet', 'unjacketed', 'hulking', 'restates', 'voting', 'ugly', 'partnership', 'facade', 'floor', 'takeover', 'asians', 'zhok', 'demented', 'materialize', 'quince', 'dreamboat', 'giovanni', 'memorized', 'dudley', 'musee', 'censorship', 'herb', 'main', 'participate', 'insatiable', 'wintered', 'our', 'thorpe', 'reposed', 'abstractive', 'zhitkov', 'frogs', 'tangled', 'versatility', 'comparative', 'filibusters', 'differentiated', 'concededly', 'favors', 'herbs', 'saps', 'precooked', 'sparing', 'insistence', 'sanctuary', 'ethos', 'apathetic', 'diagnosing', 'providing', 'unobtrusive', 'dyer', 'snuffboxes', 'hearer', 'laps', 'deftness', 'stripes', 'fractionated', 'onslaughts', 'chronically', 'tipple', 'hunt', 'discont

In [186]:
def find_palindrome_centers(words: list[str]) -> list[tuple[str, int]]:
    """
    Find all potential palindrome centers in words, excluding trivial cases.
    Returns list of (word, position) tuples where position could be center of palindrome.
    Only returns positions where there's a true palindrome opportunity (entire left substring
    matches reverse of right substring).

    Time complexity: O(N * L) where N is number of words and L is average word length
    Space complexity: O(K) where K is number of valid palindrome centers found
    """
    results = []

    for word in words:
        length = len(word)
        if length < 3:  # Skip very short words
            continue

        # For each position (excluding first two and last two characters)
        for pos in range(2, length - 2):
            # Skip center position of word
            if pos == length // 2:
                continue

            # Get entire left and right substrings
            left = word[:pos]
            right = word[pos + 1 :]

            # Check if left matches reverse of right (up to shorter length)
            min_length = min(len(left), len(right))
            if min_length > 0 and left[-min_length:] == right[:min_length][::-1]:
                results.append((word, pos))

    return results

In [187]:
results = find_palindrome_centers(vocabulary)
print(results)
print(f"{len(results)} potential palindrome centers: {results[:5]}...")

[('selflessness', 3), ('awkwardly', 2), ('repercussions', 2), ('divides', 2), ('hardened', 5), ('embroidered', 8), ('minimized', 2), ('acetate', 4), ('leveled', 2), ('reversed', 2), ('wandered', 5), ('lavallade', 2), ('interpret', 5), ('carneigie', 6), ('repertory', 2), ('pondered', 5), ('dexedrine', 2), ('reversal', 2), ('signing', 4), ('disordered', 7), ('garages', 2), ('monagan', 4), ('decedent', 2), ('saddened', 5), ('misinterpret', 8), ('analyticity', 7), ('flannagan', 6), ('reigning', 5), ('flyaway', 4), ('pacific', 4), ('parapets', 2), ('reconsidered', 9), ('minimum', 2), ('madama', 2), ('reservoirs', 2), ('paraphrases', 2), ('minimally', 2), ('sensuous', 5), ('stranahan', 6), ('wondered', 5), ('referrals', 2), ('reservations', 2), ('refers', 2), ('remodeled', 6), ('reverdy', 2), ('tendered', 5), ('paraphrasing', 2), ('stetson', 2), ('reverberation', 2), ('senese', 2), ('capacitance', 2), ('tenets', 2), ('lasalle', 2), ('deadened', 5), ('reverie', 2), ('laundered', 6), ('blunder

In [188]:
def is_palindrome(words: list[str]) -> bool:
    """Check if word sequence forms palindrome"""
    s = "".join(words)
    return s == s[::-1]

In [ ]:
@dataclass
class PalindromeFinder:
    vocabulary: set[str] = field(default_factory=set)
    prefix_cache: dict[str, set[str]] = field(default_factory=dict)
    suffix_cache: dict[str, set[str]] = field(default_factory=dict)

    def __post_init__(self) -> None:
        """Initialize caches after instance creation"""
        self._build_caches()

    def _build_caches(self) -> None:
        """Build prefix and suffix caches for efficient word lookup"""
        for word in self.vocabulary:
            for i in range(1, len(word) + 1):
                prefix, suffix = word[:i], word[-i:]
                self.prefix_cache.setdefault(prefix, set()).add(word)
                self.suffix_cache.setdefault(suffix, set()).add(word)

    @staticmethod
    def is_palindrome(words: list[str]) -> bool:
        """Check if word sequence forms palindrome"""
        s = "".join(words)
        return s == s[::-1]

    def find_matches(self, pattern: str, match_start: bool = True) -> set[str]:
        """Find all words that start/end with pattern"""
        return (
            self.prefix_cache.get(pattern, set())
            if match_start
            else self.suffix_cache.get(pattern, set())
        )

    def find_mismatch(self, words: list[str], center_pos: int) -> tuple[str, bool]:
        """
        Find what needs to be matched and on which side.
        Returns (unmatched_portion, needs_right_match)
        """
        s = "".join(words)
        left, right = s[:center_pos], s[center_pos + 1 :]

        # Find length of matching portion
        match_len = 0
        for i in range(min(len(left), len(right))):
            if left[-(i + 1)] != right[i]:
                break
            match_len = i + 1

        # Get unmatched portions
        left_unmatched = left[:-match_len] if match_len else left
        right_unmatched = right[match_len:] if match_len else right

        # Return longer unmatched portion and whether we need to match on right
        return (
            (left_unmatched, True)
            if len(left_unmatched) >= len(right_unmatched)
            else (right_unmatched, False)
        )

    def grow_palindromes(self, words: list[str], center_pos: int, depth: int = 5) -> Iterator[str]:
        """
        Recursively grow palindromes from initial words.
        Yields valid palindromes as space-separated strings.
        """
        if depth <= 0:
            return

        if self.is_palindrome(words):
            yield " ".join(words)
            return

        mismatch, needs_right = self.find_mismatch(words, center_pos)
        if not mismatch:
            return

        # Find matching words for the reversed mismatch pattern
        pattern = mismatch[::-1]
        matches = self.find_matches(pattern, match_start=needs_right)

        for word in matches:
            new_words = words + [word] if needs_right else [word] + words
            new_center = center_pos if needs_right else center_pos + len(word)

            if self.is_palindrome(new_words):
                yield " ".join(new_words)
            yield from self.grow_palindromes(new_words, new_center, depth - 1)


def filter_palindrome(palindrome: str, min_avg_length: int = 5) -> bool:
    """
    Filter palindromes based on criteria:
    - No repeated words
    - Minimum average word length
    - First word doesn't contain palindrome prefix
    """
    words = palindrome.split()

    # Check for repeated words
    if len(set(words)) != len(words):
        return False

    # Check average word length
    if sum(len(word) for word in words) / len(words) < min_avg_length:
        return False

    # Check first word for palindrome prefix
    first_word = words[0]
    return not any(PalindromeFinder.is_palindrome([first_word[:i]]) for i in range(2, 7))

In [193]:
finder = PalindromeFinder(vocabulary)
print("Finding palindromes...")

for palindrome in finder.grow_palindromes(["be"], center_pos=0, depth=6):
    if filter_palindrome(palindrome):
        print(palindrome)

Finding palindromes...
delivers levers page be gaps revels reviled
rewards levers page be gaps revels drawer
drawers levers page be gaps revels reward
diapers levers page be gaps revels repaid
lessens selfless page be gaps selflessness el
delivers lives page be gaps evils reviled
delivers rails page be gaps liars reviled
gardenia corps page be gaps procaine drag
delivery swords page be gaps drowsy reviled
delivers page be gaps reviled
delivers page be gaps reviled
delivery moors page be gaps roomy reviled
delivers moors page be gaps rooms reviled
delivers draws page be gaps wards reviled
stardel levels page be gaps levelled rats
infidel levels page be gaps levelled ifni
delivery traps page be gaps party reviled
delivers traps page be gaps parts reviled
delivered lows page be gaps wolde reviled
delivers metis page be gaps items reviled
delivered arts page be gaps trade reviled
levers rewards page be gaps drawers revel
havens rewards page be gaps drawers nevah
lagers rewards page be gaps

KeyboardInterrupt: 